# 🏥 Patient Readmission Analysis
## End-to-End Healthcare Data Analysis — No ML Required

**Objective:** Understand *who* gets readmitted within 30 days, *why*, and *what* clinical patterns signal high risk — using SQL, statistical analysis, and data visualization.

**Dataset:** [Diabetes 130-US Hospitals (1999–2008)](https://archive.ics.uci.edu/dataset/296/diabetes+130-us+hospitals+for+years+1999-2008) — UCI ML Repository

**Stack:** Python · pandas · matplotlib · seaborn · SQLite

---
### Pipeline
```
1. Data Ingestion & Cleaning
2. SQL-Based Exploratory Queries
3. Cohort Profiling — Who Gets Readmitted?
4. Statistical Analysis — Which Factors Matter?
5. Trend & Segment Analysis
6. Rule-Based Risk Scoring
7. Insights & Recommendations
```

## 📦 Step 1: Imports & Setup

In [ ]:
import pandas as pd
import numpy as np
import sqlite3
import warnings
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
from scipy import stats

warnings.filterwarnings('ignore')
sns.set_theme(style='whitegrid', palette='Set2')
pd.set_option('display.float_format', '{:.2f}'.format)
%matplotlib inline

print('✅ Libraries loaded')

## 📥 Step 2: Data Ingestion

In [ ]:
# ─── Option A: Download from UCI ───────────────────────────────────────────────
# !pip install ucimlrepo
# from ucimlrepo import fetch_ucirepo
# ds = fetch_ucirepo(id=296)
# df = ds.data.features.copy()
# df['readmitted'] = (ds.data.targets['readmitted'] == '<30').astype(int)

# ─── Option B: CSV ─────────────────────────────────────────────────────────────
# df = pd.read_csv('diabetic_data.csv')
# df['readmitted'] = (df['readmitted'] == '<30').astype(int)

# ─── Synthetic Demo Data (mirrors real dataset structure) ──────────────────────
np.random.seed(42)
n = 6000

age_groups = ['[0-10)','[10-20)','[20-30)','[30-40)','[40-50)',
              '[50-60)','[60-70)','[70-80)','[80-90)','[90-100)']
age_probs  = [0.01, 0.01, 0.03, 0.05, 0.09, 0.14, 0.21, 0.24, 0.17, 0.05]

df = pd.DataFrame({
    'encounter_id':         np.arange(1, n+1),
    'patient_nbr':          np.random.randint(10000, 99999, n),
    'race':                 np.random.choice(['Caucasian','AfricanAmerican','Hispanic','Asian','Other'], n,
                                             p=[0.75, 0.19, 0.02, 0.02, 0.02]),
    'gender':               np.random.choice(['Male','Female'], n),
    'age':                  np.random.choice(age_groups, n, p=age_probs),
    'time_in_hospital':     np.random.randint(1, 14, n),
    'num_lab_procedures':   np.random.randint(1, 130, n),
    'num_procedures':       np.random.randint(0, 7, n),
    'num_medications':      np.random.randint(1, 80, n),
    'number_outpatient':    np.random.poisson(0.3, n),
    'number_emergency':     np.random.poisson(0.2, n),
    'number_inpatient':     np.random.poisson(0.4, n),
    'number_diagnoses':     np.random.randint(1, 16, n),
    'insulin':              np.random.choice(['No','Down','Steady','Up'], n, p=[0.47,0.06,0.40,0.07]),
    'diabetesMed':          np.random.choice(['Yes','No'], n, p=[0.77,0.23]),
    'A1Cresult':            np.random.choice(['>8','>7','Norm','None'], n, p=[0.15,0.05,0.08,0.72]),
    'discharge_disposition_id': np.random.choice([1,2,3,6,11,18,25,28], n),
    'admission_source_id':  np.random.choice([1,2,3,4,5,6,7], n),
})

# Realistic target: ~11% readmission rate driven by clinical factors
logit = (
    -3.5
    + 0.08 * df['time_in_hospital']
    + 0.02 * df['num_medications']
    + 0.35 * df['number_inpatient']
    + 0.28 * df['number_emergency']
    + 0.15 * (df['diabetesMed'] == 'Yes').astype(int)
    + 0.20 * (df['A1Cresult'] == '>8').astype(int)
)
prob = 1 / (1 + np.exp(-logit))
df['readmitted'] = (np.random.rand(n) < prob).astype(int)

print(f'Records: {len(df):,}')
print(f'Readmission rate: {df["readmitted"].mean():.1%}')
df.head(3)

## 🧹 Step 3: Data Cleaning

Applied to the **real UCI dataset**. The synthetic demo data is already clean, but this step runs safely on both.

| Issue | Action |
|-------|--------|
| `?` placeholders | Replace with `NaN` |
| `weight` column (97% missing) | Drop entirely |
| Zero-variance columns | Drop (`examide`, `citoglipton`) |
| Deceased / hospice patients | Remove (can't be readmitted) |
| Duplicate patients | Keep first encounter only |
| Invalid gender entries | Remove |
| Missing categoricals | Fill with `'Unknown'` |

In [ ]:
print('=== Before Cleaning ===')
print(f'Rows: {df.shape[0]:,}  |  Columns: {df.shape[1]}')
print(f'Duplicate patient encounters: {df.duplicated(subset="patient_nbr").sum():,}')
print(f'Missing values:\n{df.isnull().sum()[df.isnull().sum() > 0]}')

In [ ]:
df_clean = df.copy()

# ── 1. Replace '?' with NaN (real dataset uses '?' for missing) ───────────────
df_clean.replace('?', np.nan, inplace=True)

# ── 2. Drop high-missingness & zero-variance columns ─────────────────────────
cols_to_drop = []

# Drop columns with >40% missing
missing_pct = df_clean.isnull().mean()
high_missing = missing_pct[missing_pct > 0.40].index.tolist()
cols_to_drop.extend(high_missing)

# Drop zero-variance columns (real dataset: examide, citoglipton always 'No')
zero_var = [c for c in df_clean.select_dtypes(include='object').columns
            if df_clean[c].nunique(dropna=True) <= 1]
cols_to_drop.extend(zero_var)

cols_to_drop = list(set(cols_to_drop))  # deduplicate
if cols_to_drop:
    df_clean.drop(columns=cols_to_drop, inplace=True)
    print(f'Dropped {len(cols_to_drop)} columns: {cols_to_drop}')

# ── 3. Remove deceased / hospice patients (discharge_disposition_id) ──────────
# IDs 11,13,14,19,20,21 = expired or hospice — cannot be readmitted
EXCLUDE_DISPOSITIONS = [11, 13, 14, 19, 20, 21]
before = len(df_clean)
df_clean = df_clean[~df_clean['discharge_disposition_id'].isin(EXCLUDE_DISPOSITIONS)]
print(f'Removed {before - len(df_clean):,} deceased/hospice patients')

# ── 4. Remove invalid gender ──────────────────────────────────────────────────
before = len(df_clean)
df_clean = df_clean[df_clean['gender'].isin(['Male', 'Female'])]
print(f'Removed {before - len(df_clean):,} invalid gender rows')

# ── 5. Keep only first encounter per patient (avoid data leakage) ─────────────
before = len(df_clean)
df_clean = df_clean.sort_values('encounter_id').drop_duplicates(subset='patient_nbr', keep='first')
print(f'Removed {before - len(df_clean):,} duplicate patient encounters (kept first visit)')

# ── 6. Fill remaining missing categoricals with 'Unknown' ─────────────────────
cat_cols = df_clean.select_dtypes(include='object').columns
df_clean[cat_cols] = df_clean[cat_cols].fillna('Unknown')

print(f'\n=== After Cleaning ===')
print(f'Rows: {df_clean.shape[0]:,}  |  Columns: {df_clean.shape[1]}')
print(f'Remaining missing values: {df_clean.isnull().sum().sum()}')
print(f'Readmission rate (post-clean): {df_clean["readmitted"].mean():.1%}')

In [ ]:
# ── Cleaning Summary Visual ───────────────────────────────────────────────────
fig, axes = plt.subplots(1, 3, figsize=(16, 4))
fig.suptitle('Data Cleaning Summary', fontsize=13, fontweight='bold')

# Before vs After row count
ax = axes[0]
bars = ax.bar(['Before Cleaning', 'After Cleaning'],
              [len(df), len(df_clean)],
              color=['#90CAF9', '#1B4F8A'], edgecolor='white')
ax.set_title('Row Count', fontweight='bold')
ax.set_ylabel('Records')
for bar, val in zip(bars, [len(df), len(df_clean)]):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 20,
            f'{val:,}', ha='center', va='bottom', fontweight='bold')

# Missing values heatmap (before)
ax = axes[1]
missing_before = df.isnull().sum()
missing_before = missing_before[missing_before > 0].sort_values(ascending=True)
if len(missing_before) > 0:
    ax.barh(missing_before.index, missing_before.values, color='#EF9A9A', edgecolor='white')
    ax.set_title('Missing Values (Before)', fontweight='bold')
    ax.set_xlabel('Count')
else:
    ax.text(0.5, 0.5, 'No missing values\n(synthetic data)', ha='center', va='center',
            transform=ax.transAxes, fontsize=11, color='grey')
    ax.set_title('Missing Values (Before)', fontweight='bold')

# Steps applied breakdown
ax = axes[2]
steps = ['? → NaN', 'Drop high-\nmissing cols', 'Remove\ndeceased', 'Remove\ninvalid gender', 'Deduplicate\npatients']
removed = [
    0,
    len(cols_to_drop) if cols_to_drop else 0,
    len(df[df['discharge_disposition_id'].isin(EXCLUDE_DISPOSITIONS)]),
    len(df[~df['gender'].isin(['Male','Female'])]),
    df.duplicated(subset='patient_nbr').sum()
]
colors = ['#B0BEC5','#EF9A9A','#FFCC80','#CE93D8','#80CBC4']
ax.barh(steps, removed, color=colors, edgecolor='white')
ax.set_title('Records Removed per Step', fontweight='bold')
ax.set_xlabel('Records Removed')
for i, val in enumerate(removed):
    ax.text(val + 0.5, i, str(val), va='center', fontsize=9)

plt.tight_layout()
plt.savefig('/home/claude/portfolio2/healthcare-readmission/data_cleaning_summary.png', dpi=150, bbox_inches='tight')
plt.show()

# Use cleaned dataframe going forward
df = df_clean.copy()
print('\n✅ df now points to the cleaned dataset — all subsequent steps use clean data')

## 🗄️ Step 4: SQL Analysis

In [ ]:
conn = sqlite3.connect(':memory:')
df.to_sql('encounters', conn, if_exists='replace', index=False)

# ── Overview ──────────────────────────────────────────────────────────────────
overview = pd.read_sql_query("""
    SELECT
        COUNT(*)                              AS total_encounters,
        COUNT(DISTINCT patient_nbr)           AS unique_patients,
        SUM(readmitted)                       AS total_readmissions,
        ROUND(AVG(readmitted)*100, 1)         AS readmission_rate_pct,
        ROUND(AVG(time_in_hospital), 1)       AS avg_los_days,
        ROUND(AVG(num_medications), 1)        AS avg_medications
    FROM encounters
""", conn)
print('=== Dataset Overview ===')
print(overview.T.to_string(header=False))

In [ ]:
# ── Readmission by age ────────────────────────────────────────────────────────
by_age = pd.read_sql_query("""
    SELECT
        age,
        COUNT(*)                              AS encounters,
        SUM(readmitted)                       AS readmissions,
        ROUND(AVG(readmitted)*100, 1)         AS rate_pct,
        ROUND(AVG(time_in_hospital), 1)       AS avg_los
    FROM encounters
    GROUP BY age
    ORDER BY age
""", conn)
print('=== Readmission Rate by Age Group ===')
print(by_age.to_string(index=False))

In [ ]:
# ── Clinical profile: readmitted vs not ───────────────────────────────────────
profile = pd.read_sql_query("""
    SELECT
        CASE WHEN readmitted=1 THEN 'Readmitted' ELSE 'Not Readmitted' END AS cohort,
        COUNT(*)                              AS n,
        ROUND(AVG(time_in_hospital), 2)       AS avg_los,
        ROUND(AVG(num_medications), 2)        AS avg_meds,
        ROUND(AVG(number_inpatient), 2)       AS avg_prior_inpatient,
        ROUND(AVG(number_emergency), 2)       AS avg_prior_emergency,
        ROUND(AVG(number_diagnoses), 2)       AS avg_diagnoses
    FROM encounters
    GROUP BY readmitted
""", conn)
print('=== Clinical Profile Comparison ===')
print(profile.to_string(index=False))

In [ ]:
# ── A1C and medication analysis ───────────────────────────────────────────────
a1c = pd.read_sql_query("""
    SELECT
        A1Cresult,
        insulin,
        COUNT(*)                              AS n,
        ROUND(AVG(readmitted)*100, 1)         AS readmit_pct,
        ROUND(AVG(time_in_hospital), 1)       AS avg_los
    FROM encounters
    GROUP BY A1Cresult, insulin
    ORDER BY readmit_pct DESC
    LIMIT 12
""", conn)
print('=== A1C × Insulin Readmission Rates ===')
print(a1c.to_string(index=False))

In [ ]:
# ── High utilization cohort ───────────────────────────────────────────────────
high_util = pd.read_sql_query("""
    SELECT
        CASE
            WHEN (number_inpatient + number_emergency) = 0 THEN '0 prior visits'
            WHEN (number_inpatient + number_emergency) = 1 THEN '1 prior visit'
            WHEN (number_inpatient + number_emergency) BETWEEN 2 AND 3 THEN '2–3 prior visits'
            ELSE '4+ prior visits'
        END AS prior_visits_bucket,
        COUNT(*)                              AS n,
        ROUND(AVG(readmitted)*100, 1)         AS readmit_pct,
        ROUND(AVG(time_in_hospital), 1)       AS avg_los
    FROM encounters
    GROUP BY prior_visits_bucket
    ORDER BY readmit_pct DESC
""", conn)
print('=== Readmission by Prior Utilization ===')
print(high_util.to_string(index=False))

## 📊 Step 5: Cohort EDA — Who Gets Readmitted?

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(18, 11))
fig.suptitle('Patient Readmission — Exploratory Analysis', fontsize=16, fontweight='bold', y=0.98)

palette = {'Not Readmitted': '#4CAF50', 'Readmitted': '#E53935'}

# Label helper
df['status'] = df['readmitted'].map({0: 'Not Readmitted', 1: 'Readmitted'})

# 1. Readmission rate by age group
ax = axes[0, 0]
age_rate = df.groupby('age')['readmitted'].mean().mul(100)
colors = plt.cm.YlOrRd(age_rate.values / age_rate.max())
bars = ax.bar(range(len(age_rate)), age_rate.values, color=colors, edgecolor='white', linewidth=0.8)
ax.set_xticks(range(len(age_rate)))
ax.set_xticklabels(age_rate.index, rotation=45, ha='right', fontsize=8)
ax.set_title('Readmission Rate by Age Group', fontweight='bold')
ax.set_ylabel('Readmission Rate (%)')
for bar, val in zip(bars, age_rate.values):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.2, f'{val:.1f}%',
            ha='center', va='bottom', fontsize=7.5)

# 2. Length of stay — overlapping histograms
ax = axes[0, 1]
for label, color in [('Not Readmitted','#4CAF50'), ('Readmitted','#E53935')]:
    subset = df[df['status'] == label]['time_in_hospital']
    ax.hist(subset, bins=13, alpha=0.6, color=color, label=label, edgecolor='white')
ax.axvline(df[df['readmitted']==1]['time_in_hospital'].mean(), color='#E53935', linestyle='--', lw=2)
ax.axvline(df[df['readmitted']==0]['time_in_hospital'].mean(), color='#4CAF50', linestyle='--', lw=2)
ax.set_title('Length of Stay Distribution', fontweight='bold')
ax.set_xlabel('Days in Hospital')
ax.legend(fontsize=8)

# 3. Readmission rate by A1C result
ax = axes[0, 2]
a1c_rate = df.groupby('A1Cresult')['readmitted'].mean().mul(100).sort_values(ascending=True)
a1c_colors = ['#4CAF50' if v < 12 else '#FF9800' if v < 15 else '#E53935' for v in a1c_rate.values]
ax.barh(a1c_rate.index, a1c_rate.values, color=a1c_colors, edgecolor='white')
ax.set_title('Readmission Rate by A1C Result', fontweight='bold')
ax.set_xlabel('Readmission Rate (%)')
for i, val in enumerate(a1c_rate.values):
    ax.text(val + 0.1, i, f'{val:.1f}%', va='center', fontsize=9)

# 4. Prior inpatient visits vs readmission
ax = axes[1, 0]
prior_rate = df.groupby('number_inpatient')['readmitted'].agg(['mean','count'])
prior_rate = prior_rate[prior_rate['count'] >= 20].head(8)
prior_rate['mean'] = prior_rate['mean'] * 100
ax.bar(prior_rate.index, prior_rate['mean'], color='#2196F3', edgecolor='white')
ax.set_title('Readmission Rate by Prior Inpatient Visits', fontweight='bold')
ax.set_xlabel('Number of Prior Inpatient Stays')
ax.set_ylabel('Readmission Rate (%)')

# 5. Medications distribution by readmission status
ax = axes[1, 1]
bp_data = [df[df['readmitted']==0]['num_medications'].values,
           df[df['readmitted']==1]['num_medications'].values]
bp = ax.boxplot(bp_data, patch_artist=True, labels=['Not\nReadmitted','Readmitted'],
                medianprops=dict(color='black', linewidth=2))
for patch, color in zip(bp['boxes'], ['#4CAF50','#E53935']):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)
ax.set_title('Medications by Readmission Status', fontweight='bold')
ax.set_ylabel('Number of Medications')

# 6. Insulin usage heatmap
ax = axes[1, 2]
heat = df.groupby(['insulin','diabetesMed'])['readmitted'].mean().mul(100).unstack()
sns.heatmap(heat, ax=ax, annot=True, fmt='.1f', cmap='YlOrRd',
            linewidths=0.5, cbar_kws={'label': 'Readmission %'})
ax.set_title('Readmission % — Insulin × Diabetes Med', fontweight='bold')
ax.set_xlabel('Diabetes Medication')
ax.set_ylabel('Insulin')

plt.tight_layout()
plt.savefig('/home/claude/portfolio2/healthcare-readmission/eda_charts.png', dpi=150, bbox_inches='tight')
plt.show()
print('✅ EDA saved')

## 📐 Step 6: Statistical Analysis — Which Factors Are Significant?

In [ ]:
readmit_yes = df[df['readmitted'] == 1]
readmit_no  = df[df['readmitted'] == 0]

numeric_vars = [
    ('time_in_hospital',   'Length of Stay'),
    ('num_medications',    'Num Medications'),
    ('number_inpatient',   'Prior Inpatient Visits'),
    ('number_emergency',   'Prior Emergency Visits'),
    ('num_lab_procedures', 'Lab Procedures'),
    ('number_diagnoses',   'Number of Diagnoses'),
]

print(f'{"Variable":<28} {"Mean (Readmit)":>15} {"Mean (No Readmit)":>18} {"Diff %":>8} {"p-value":>10} {"Significant":>12}')
print('-' * 95)

stat_results = []
for col, label in numeric_vars:
    m1  = readmit_yes[col].mean()
    m0  = readmit_no[col].mean()
    pct = (m1 - m0) / m0 * 100
    t, p = stats.ttest_ind(readmit_yes[col], readmit_no[col])
    sig  = '✅ Yes' if p < 0.05 else '❌ No'
    stat_results.append({'Variable': label, 'Readmitted Mean': m1, 'Not Readmitted Mean': m0,
                         'Diff %': pct, 'p-value': p})
    print(f'{label:<28} {m1:>15.2f} {m0:>18.2f} {pct:>7.1f}% {p:>10.4f} {sig:>12}')

In [ ]:
# ── Chi-square tests for categorical variables ────────────────────────────────
from scipy.stats import chi2_contingency

cat_vars = ['A1Cresult', 'insulin', 'diabetesMed', 'gender', 'race']
print(f'{"Variable":<20} {"Chi2":>10} {"p-value":>10} {"Cramers V":>10} {"Significant":>12}')
print('-' * 65)

for col in cat_vars:
    ct   = pd.crosstab(df[col], df['readmitted'])
    chi2, p, dof, _ = chi2_contingency(ct)
    n    = ct.values.sum()
    cramer = np.sqrt(chi2 / (n * (min(ct.shape) - 1)))
    sig  = '✅ Yes' if p < 0.05 else '❌ No'
    print(f'{col:<20} {chi2:>10.2f} {p:>10.4f} {cramer:>10.3f} {sig:>12}')

## 📈 Step 7: Segment & Trend Analysis

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 11))
fig.suptitle('Segment & Trend Analysis', fontsize=15, fontweight='bold')

# 1. Diverging bar: % difference from average readmission rate
ax = axes[0, 0]
base_rate = df['readmitted'].mean() * 100
age_diff  = df.groupby('age')['readmitted'].mean().mul(100) - base_rate
colors    = ['#E53935' if v > 0 else '#4CAF50' for v in age_diff.values]
ax.barh(age_diff.index, age_diff.values, color=colors, edgecolor='white')
ax.axvline(0, color='black', lw=1.5)
ax.set_title(f'Readmission Rate vs Average ({base_rate:.1f}%) by Age', fontweight='bold')
ax.set_xlabel('Percentage Point Difference from Average')

# 2. Stacked bar — prior utilization bucket
ax = axes[0, 1]
df['util_bucket'] = pd.cut(
    df['number_inpatient'] + df['number_emergency'],
    bins=[-1, 0, 1, 3, 100],
    labels=['None', '1 visit', '2–3 visits', '4+ visits']
)
util_ct = df.groupby(['util_bucket','readmitted']).size().unstack(fill_value=0)
util_pct = util_ct.div(util_ct.sum(axis=1), axis=0) * 100
util_pct.plot(kind='bar', ax=ax, color=['#4CAF50','#E53935'], edgecolor='white', stacked=True)
ax.set_title('Readmission Rate by Prior Utilization', fontweight='bold')
ax.set_xlabel('')
ax.set_ylabel('% of Patients')
ax.legend(['Not Readmitted','Readmitted'], loc='lower right')
ax.tick_params(axis='x', rotation=0)
for i, (idx, row) in enumerate(util_pct.iterrows()):
    ax.text(i, row[1]/2, f'{row[1]:.1f}%', ha='center', va='center',
            color='white', fontweight='bold', fontsize=10)

# 3. Length of stay vs readmission rate — rolling window
ax = axes[1, 0]
los_rate = df.groupby('time_in_hospital')['readmitted'].agg(['mean','count'])
los_rate = los_rate[los_rate['count'] >= 30]
ax.plot(los_rate.index, los_rate['mean']*100, 'o-', color='#2196F3', lw=2, markersize=7)
ax.fill_between(los_rate.index, los_rate['mean']*100, alpha=0.2, color='#2196F3')
ax.axhline(base_rate, color='grey', linestyle='--', label=f'Average ({base_rate:.1f}%)')
ax.set_title('Readmission Rate by Length of Stay', fontweight='bold')
ax.set_xlabel('Days in Hospital')
ax.set_ylabel('Readmission Rate (%)')
ax.legend()

# 4. Medications × diagnoses — bubble chart
ax = axes[1, 1]
agg = df.groupby(['num_medications', 'number_diagnoses']).agg(
    readmit_rate=('readmitted','mean'),
    count=('readmitted','count')
).reset_index()
agg = agg[agg['count'] >= 10]
sc = ax.scatter(
    agg['num_medications'], agg['number_diagnoses'],
    c=agg['readmit_rate']*100, s=agg['count']/2,
    cmap='YlOrRd', alpha=0.7, edgecolors='grey', linewidths=0.5
)
plt.colorbar(sc, ax=ax, label='Readmission Rate (%)')
ax.set_title('Readmission Rate by Medications × Diagnoses\n(bubble size = patient count)', fontweight='bold')
ax.set_xlabel('Number of Medications')
ax.set_ylabel('Number of Diagnoses')

plt.tight_layout()
plt.savefig('/home/claude/portfolio2/healthcare-readmission/segment_analysis.png', dpi=150, bbox_inches='tight')
plt.show()
print('✅ Segment charts saved')

## 🚦 Step 8: Rule-Based Risk Scoring

In [ ]:
"""
Rule-Based Risk Score (no ML required)
Each rule is grounded in the EDA and statistical analysis above.
Score range: 0–10 → mapped to 4 risk tiers.
"""

def compute_risk_score(row):
    score = 0
    # Prior utilization — strongest signal from EDA
    score += min(row['number_inpatient'] * 2, 4)     # up to 4 pts
    score += min(row['number_emergency'] * 1, 2)     # up to 2 pts
    # Length of stay
    if row['time_in_hospital'] >= 7:  score += 1
    if row['time_in_hospital'] >= 10: score += 1
    # Glycemic control
    if row['A1Cresult'] == '>8':      score += 1
    # Polypharmacy
    if row['num_medications'] >= 20:  score += 1
    return min(score, 10)  # cap at 10

df['risk_score'] = df.apply(compute_risk_score, axis=1)

df['risk_tier'] = pd.cut(
    df['risk_score'],
    bins=[-1, 1, 3, 6, 10],
    labels=['Low (0–1)','Medium (2–3)','High (4–6)','Critical (7–10)']
)

# Validate — does score correlate with actual readmission?
validation = df.groupby('risk_tier').agg(
    patients=('readmitted','count'),
    actual_readmission_rate=('readmitted','mean'),
    avg_risk_score=('risk_score','mean')
)
validation['actual_readmission_rate'] = (validation['actual_readmission_rate'] * 100).round(1)
validation['avg_risk_score'] = validation['avg_risk_score'].round(2)

print('=== Risk Tier Validation ===')
print(validation.to_string())

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
fig.suptitle('Risk Scoring — Validation & Distribution', fontsize=14, fontweight='bold')

tier_colors = ['#4CAF50','#FFC107','#FF5722','#B71C1C']
tiers = validation.index.tolist()

# 1. Actual readmission rate per tier (validation bar)
ax = axes[0]
rates = validation['actual_readmission_rate'].values
bars = ax.bar(tiers, rates, color=tier_colors, edgecolor='white')
ax.axhline(df['readmitted'].mean()*100, color='grey', linestyle='--', label='Overall avg')
ax.set_title('Actual Readmission Rate\nby Risk Tier (Validation)', fontweight='bold')
ax.set_ylabel('Actual Readmission Rate (%)')
ax.tick_params(axis='x', rotation=15)
ax.legend(fontsize=9)
for bar, val in zip(bars, rates):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.3, f'{val:.1f}%',
            ha='center', va='bottom', fontweight='bold')

# 2. Patient volume per tier
ax = axes[1]
counts = validation['patients'].values
wedges, texts, autotexts = ax.pie(
    counts, labels=tiers, colors=tier_colors,
    autopct='%1.1f%%', startangle=90, pctdistance=0.8
)
ax.set_title('Patient Distribution\nby Risk Tier', fontweight='bold')

# 3. Risk score distribution
ax = axes[2]
ax.hist(df['risk_score'], bins=11, range=(-0.5,10.5), color='#2196F3', edgecolor='white', alpha=0.8)
ax.axvline(1.5, color='#FFC107', lw=2, linestyle='--', label='Low→Medium')
ax.axvline(3.5, color='#FF5722', lw=2, linestyle='--', label='Medium→High')
ax.axvline(6.5, color='#B71C1C', lw=2, linestyle='--', label='High→Critical')
ax.set_title('Risk Score Distribution\nwith Tier Thresholds', fontweight='bold')
ax.set_xlabel('Risk Score (0–10)')
ax.set_ylabel('Patient Count')
ax.legend(fontsize=8)

plt.tight_layout()
plt.savefig('/home/claude/portfolio2/healthcare-readmission/risk_scoring.png', dpi=150, bbox_inches='tight')
plt.show()

## 💡 Step 9: Insights & Recommendations

In [ ]:
print('=' * 60)
print('KEY FINDINGS')
print('=' * 60)

# Finding 1: Utilization lift
no_prior = df[df['number_inpatient']==0]['readmitted'].mean()*100
hi_prior = df[df['number_inpatient']>=3]['readmitted'].mean()*100
print(f'\n1. Prior Inpatient Impact:')
print(f'   No prior inpatient stays → {no_prior:.1f}% readmission rate')
print(f'   3+ prior inpatient stays → {hi_prior:.1f}% readmission rate ({hi_prior/no_prior:.1f}x higher)')

# Finding 2: A1C impact
no_a1c  = df[df['A1Cresult']=='None']['readmitted'].mean()*100
hi_a1c  = df[df['A1Cresult']=='>8']['readmitted'].mean()*100
print(f'\n2. A1C Result Impact:')
print(f'   A1C not measured → {no_a1c:.1f}% readmission rate')
print(f'   A1C > 8 (poor control) → {hi_a1c:.1f}% readmission rate')

# Finding 3: Long LOS
short_los = df[df['time_in_hospital']<=3]['readmitted'].mean()*100
long_los  = df[df['time_in_hospital']>=10]['readmitted'].mean()*100
print(f'\n3. Length of Stay:')
print(f'   LOS ≤ 3 days → {short_los:.1f}% readmission rate')
print(f'   LOS ≥ 10 days → {long_los:.1f}% readmission rate')

# Finding 4: Risk tier breakdown
critical_n = (df['risk_tier']=='Critical (7–10)').sum()
critical_r = df[df['risk_tier']=='Critical (7–10)']['readmitted'].mean()*100
print(f'\n4. Critical-Tier Patients:')
print(f'   {critical_n:,} patients ({critical_n/len(df):.1%} of cohort) in Critical tier')
print(f'   Actual readmission rate: {critical_r:.1f}%')

print(f'\n{"-"*60}')
print('RECOMMENDATIONS')
print('-' * 60)
recs = [
    '1. Focus discharge planning on patients with 2+ prior inpatient stays',
    '2. Make A1C testing standard — untested patients are an undetected risk group',
    '3. Assign care coordinators to all Critical-tier patients before discharge',
    '4. Implement 48-hour post-discharge phone follow-up for High & Critical tiers',
    '5. Review insulin protocols for patients with frequent emergency visits',
    '6. Embed the risk score in the EHR discharge workflow as a decision-support alert',
]
for r in recs:
    print(f'  {r}')

# Export scored patient list for dashboard
export_cols = ['encounter_id','patient_nbr','age','gender','time_in_hospital',
               'number_inpatient','number_emergency','num_medications','A1Cresult',
               'risk_score','risk_tier','readmitted']
df[export_cols].to_csv('/home/claude/portfolio2/healthcare-readmission/scored_patients.csv', index=False)
print('\n✅ Scored patient list exported → scored_patients.csv (connect to Tableau / Power BI)')

---
## 📁 Output Files
| File | Use |
|------|-----|
| `eda_charts.png` | EDA visualizations |
| `segment_analysis.png` | Trend & segment charts |
| `risk_scoring.png` | Risk tier validation charts |
| `scored_patients.csv` | Connect to Tableau / Power BI dashboard |

> **No machine learning was used.** All insights are derived from SQL queries, descriptive statistics, t-tests, chi-square tests, and a transparent rule-based scoring system — fully auditable by clinical staff.